### 모델링

In [ ]:
## 필수요소 패키지 설치
import pandas as pd
import numpy as np
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize  
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
############################### 파일 불러오기#########################################
# 트레인 데이터
train_data = pd.read_csv('train_data.csv')
print(train_data)
print('전체 리뷰 개수 :',len(train_data)) 

# reviews 에 있는 문자열 모두 소문자로 변경 (대문자가 있을 경우, 명사가 대명사로 인식되는 오류 발생)
train_data['reviews'] = train_data['reviews'].str.lower()
# 긍정 부정 점수 표기
train_data['label'] = (train_data['label'] > 3) * 1
# 이모티콘 제거
tot_list = list(train_data['reviews'])
new_list = []
for i in tot_list :
    new_list.append(re.sub("[^\s!-~]+", '', i))
train_data['reviews'] = new_list
# reviews 열에서 중복인 내용이 있다면 중복 제거
train_data.drop_duplicates(subset=['reviews'], inplace=True)
# 공백 제거
train_data.dropna(inplace = True)
print(train_data)
print('전체 리뷰 개수 :',len(train_data))


# 테스트 데이터
test_data = pd.read_csv('test_data.csv')
print(test_data)
print('전체 리뷰 개수 :',len(test_data)) 

# reviews 에 있는 문자열 모두 소문자로 변경 (대문자가 있을 경우, 명사가 대명사로 인식되는 오류 발생)
test_data['reviews'] = test_data['reviews'].str.lower()

# 긍정 부정 점수 표기
test_data['label'] = (test_data['label'] > 3) * 1
# 이모티콘 제거
tot_list = list(test_data['reviews'])
new_list = []
for i in tot_list :
    new_list.append(re.sub("[^\s!-~]+", '', i))
test_data['reviews'] = new_list
# reviews 열에서 중복인 내용이 있다면 중복 제거
test_data.drop_duplicates(subset=['reviews'], inplace=True)
# 공백 제거
test_data.dropna(inplace = True)
print(test_data)
print('전체 리뷰 개수 :',len(test_data))

In [ ]:
############################### 데이터 토큰화 + 수치화 #########################################

# 길이가 2이하인 단어는 제거 (길이가 짧은 단어 제거)
train_data['reviews'] = train_data['reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# token화 및 불용어 제거
train_data['tokenized'] = train_data['reviews'].apply(word_tokenize)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x:[item for item in x if item not in stopwords.words('english')])

# 긍정 부정 단어 구분
negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)

# 정수 인코딩 단어에 번호를 붙이는 작업
X_train = train_data['tokenized'].values
y_train = train_data['label'].values 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 2                                 # 희귀단어 -> 2번이상 사용이 되지 않은 단어 
total_cnt = len(tokenizer.word_index)         # 단어 개수
rare_cnt = 0                                  # 등장 빈도가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0                                # 훈련데이터 전체 단어 빈도수 총합
rare_freq = 0                                 # 등장 빈도수가 threshold보다 작은 단어의 빈도수의 총합

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 빈도수가 threshold보다 작을경우
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

# 전체 단어 개수 Total_cnt 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2

# 정수 변환 완료
tokenizer = Tokenizer(vocab_size, oov_token = 'oov')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)

def len_length (mex_len, overlap_list):
    cnt = 0
    for a in overlap_list:
        if(len(a) <= max_len):
            cnt = cnt + 1

max_len = 100
len_length(max_len, X_train)

X_train = pad_sequences(X_train, maxlen = max_len)

In [ ]:
############################### 데이터 토큰화 + 수치화 #########################################

# 길이가 2이하인 단어는 제거 (길이가 짧은 단어 제거)
test_data['reviews'] = test_data['reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# token화 및 불용어 제거
test_data['tokenized'] = test_data['reviews'].apply(word_tokenize)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x:[item for item in x if item not in stopwords.words('english')])

# 긍정 부정 단어 구분
negative_words = np.hstack(test_data[test_data.label == 0]['tokenized'].values)
positive_words = np.hstack(test_data[test_data.label == 1]['tokenized'].values)

# 정수 인코딩 단어에 번호를 붙이는 작업
X_test = test_data['tokenized'].values
y_test = test_data['label'].values 

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_test)

threshold = 2                                 # 희귀단어 -> 2번이상 사용이 되지 않은 단어 
total_cnt = len(tokenizer.word_index)         # 단어 개수
rare_cnt = 0                                  # 등장 빈도가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0                                # 훈련데이터 전체 단어 빈도수 총합
rare_freq = 0                                 # 등장 빈도수가 threshold보다 작은 단어의 빈도수의 총합

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 빈도수가 threshold보다 작을경우
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

# 전체 단어 개수 Total_cnt 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2

# 정수 변환 완료
tokenizer = Tokenizer(vocab_size, oov_token = 'oov')
tokenizer.fit_on_texts(X_test)
X_test = tokenizer.texts_to_sequences(X_test)

def len_length (mex_len, overlap_list):
    cnt = 0
    for a in overlap_list:
        if(len(a) <= max_len):
            cnt = cnt + 1

max_len = 100
len_length(max_len, X_test)

X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
# 머신러닝 돌리기 위해 실수화
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
############################### 모델링 #########################################

from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

model = Sequential()
# 단어집합의 크기, 100차원의 임베딩 벡터 표현
model.add(Embedding(100000, 100)) 
model.add(GRU(256))
model.add(Dense(1, activation = 'sigmoid'))

# EarlyStopping은 너무 많은 Epoch로 인한 Overfitting을 방지하기 위해 설정
# es = Performance measure를 최소화 하고 verbose가 발생한 지점을 출력한다. 
# 증가하지 않는 epoch(patience)를 4번 허용한다.
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 4) 
# ModelCheckpoint는 작업중 Callback을 할경우 최적의 모델을 선택 할수 있도록 한다.
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', made = 'max', 
                     verbose = 1, save_best_only = True)  

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
hist = model.fit(X_train, y_train, epochs = 10, verbose = 1, callbacks = [es, mc], 
                 batch_size = 100, validation_data = (X_test, y_test))

In [ ]:
# 저장된 모델 가져와서 예측
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

### 함수화

In [1]:
############################ 함수 ###############################
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize  
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def sentiment_predict(total_data) :
    loaded_model = load_model('best_model.h5')   
    total_data.dropna(inplace = True)
    
    # 길이가 2이하인 단어는 제거 (길이가 짧은 단어 제거)
    total_data['comment'] = total_data['comment'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

    # token화 및 불용어 제거
    total_data['tokenized'] = total_data['comment'].apply(word_tokenize)
    total_data['tokenized'] = total_data['tokenized'].apply(lambda x:[item for item in x if item not in stopwords.words('english')])
    X_train = total_data['tokenized'].values
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)

    threshold = 2                                 # 희귀단어 -> 2번이상 사용이 되지 않은 단어 
    total_cnt = len(tokenizer.word_index)         # 단어 개수
    rare_cnt = 0                                  # 등장 빈도가 threshold보다 작은 단어의 개수를 카운트
    total_freq = 0                                # 훈련데이터 전체 단어 빈도수 총합
    rare_freq = 0                                 # 등장 빈도수가 threshold보다 작은 단어의 빈도수의 총합

    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        # 단어의 빈도수가 threshold보다 작을경우
        if(value < threshold):
            rare_cnt = rare_cnt + 1
            rare_freq = rare_freq + value

    # 전체 단어 개수 Total_cnt 중 빈도수 2이하인 단어 개수는 제거.
    # 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
    vocab_size = total_cnt - rare_cnt + 2

    # 정수 변환 완료
    tokenizer = Tokenizer(vocab_size, oov_token = 'oov')
    tokenizer.fit_on_texts(X_train)
    
    result = ''

    # 채널별 나누기 (기준값은 host : 유튜브명)
    host_li = list(set(total_data['host']))
    video_li = []
    for i in host_li :
        video_li.append(list(set(total_data[total_data['host'] == i]['url'])))

    p_pct = []
    n_pct = []
    host = []
    video = []
    sent_len = []

    # All - All    
    lis = list(total_data['comment'])
    p_reviews_cnt = 0
    n_reviews_cnt = 0

    for i in lis :
        new_sentence = i

        new_sentence = word_tokenize(new_sentence)
        # 불용어 제거
        new_sentence = [word for word in new_sentence if not word in stopwords.words('english')] 
        # 정수 인코딩
        encoded = tokenizer.texts_to_sequences([new_sentence]) 
        # 패딩
        pad_new = pad_sequences(encoded, maxlen = 100) 
        # 예측
        score = float(loaded_model.predict(pad_new)) 

        if(score > 0.5):
            p_reviews_cnt += 1
        else:
            n_reviews_cnt += 1

    p_pct.append(int(p_reviews_cnt / len(lis) * 100))
    n_pct.append(int(n_reviews_cnt / len(lis) * 100))
    host.append('All')
    video.append('All')
    sent_len.append(len(lis))


    # Channel - All
    for i in host_li :
        lis = list(total_data[total_data['host'] == i]['comment'])
        p_reviews_cnt = 0
        n_reviews_cnt = 0

        for j in range(len(lis)) :
            new_sentence = lis[j]
            new_sentence = word_tokenize(new_sentence)
            # 불용어 제거
            new_sentence = [word for word in new_sentence if not word in stopwords.words('english')] 
            # 정수 인코딩
            encoded = tokenizer.texts_to_sequences([new_sentence]) 
            # 패딩
            pad_new = pad_sequences(encoded, maxlen = 100) 
            # 예측
            score = float(loaded_model.predict(pad_new)) 

            if(score > 0.5):
                p_reviews_cnt += 1
            else:
                n_reviews_cnt += 1

        p_pct.append(int(p_reviews_cnt / len(lis) * 100))
        n_pct.append(int(n_reviews_cnt / len(lis) * 100))
        host.append(i)
        video.append('All')
        sent_len.append(len(lis))

    # Channel - Video
    for k in video_li :
        for m in range(len(k)) :
            this_host = list(set(total_data[total_data['url'] == k[m]]['host']))
            host.append(this_host[0])
        for m in k :

            lis = list(total_data[total_data['url'] == m]['comment'])
            p_reviews_cnt = 0
            n_reviews_cnt = 0
            for n in range(len(lis)) :
                new_sentence = lis[n]
                new_sentence = word_tokenize(new_sentence)
                # 불용어 제거
                new_sentence = [word for word in new_sentence if not word in stopwords.words('english')] 
                # 정수 인코딩
                encoded = tokenizer.texts_to_sequences([new_sentence]) 
                # 패딩
                pad_new = pad_sequences(encoded, maxlen = 100) 
                # 예측
                score = float(loaded_model.predict(pad_new)) 

                if(score > 0.5):
                    p_reviews_cnt += 1
                else:
                    n_reviews_cnt += 1

            p_pct.append(int(p_reviews_cnt / len(lis) * 100))
            n_pct.append(100 - int(p_reviews_cnt / len(lis) * 100))
            video.append(m)
            sent_len.append(len(lis))

    df = pd.DataFrame({'host' : host, 
                       'video' : video, 
                       'p_pct' : p_pct, 
                       'n_pct' : n_pct})
    return df

In [ ]:
# 실제 유튜브 데이터로 예측 후 결과 csv저장
baking = pd.read_csv('baking.csv')
df = sentiment_predict(baking)
df.to_csv('result_baking.csv', index = False, encoding = 'utf-8')